<a href="https://colab.research.google.com/github/jlerapatty/helmet_detection/blob/main/HelmetDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install roboflow ultralytics --quiet

from roboflow import Roboflow
from ultralytics import YOLO
import json, os

In [4]:
rf = Roboflow(api_key="97BKSCBw2Drbv6qpGq5J")

project = rf.workspace("jo-i0yms").project("helmet-detection-shw28")
dataset = project.version(2).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [5]:
model = YOLO("yolov8n.pt")

# Jalankan training
results = model.train(
    data=dataset.location + "/data.yaml",
    epochs=30,
    imgsz=640,
    batch=8,
    name="helmet_detector",
    project="runs/train"
)

Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Helmet-Detection-2/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=helmet_detector3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.

In [6]:
best_model_path = "runs/train/helmet_detector/weights/best.pt"
if not os.path.exists(best_model_path):
    raise FileNotFoundError(f"File model tidak ditemukan: {best_model_path}")

model = YOLO(best_model_path)
metrics = model.val()
print("\nEvaluasi selesai!")

Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1663.0±858.5 MB/s, size: 126.7 KB)
val: Scanning /content/Helmet-Detection-2/valid/labels.cache... 11 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 11/11 11.0Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 0.4it/s 2.7s
                   all         11         19      0.787      0.762      0.832      0.449
                helmet          4          4      0.721       0.75      0.773      0.451
             no_helmet          9         15      0.853      0.774       0.89      0.447
Speed: 1.8ms preprocess, 197.2ms inference, 0.0ms loss, 39.0ms postprocess per image
Results saved to /content/runs/detect/val3

✅ Evaluasi selesai!


In [7]:
metrics_dict = metrics.results_dict if hasattr(metrics, "results_dict") else {}

output_dir = "runs/train/helmet_detector"
os.makedirs(output_dir, exist_ok=True)

metrics_path = os.path.join(output_dir, "metrics.json")
with open(metrics_path, "w") as f:
    json.dump(metrics_dict, f, indent=4)

print(f"File metrics.json disimpan di: {metrics_path}")

✅ File metrics.json disimpan di: runs/train/helmet_detector/metrics.json


In [8]:
with open(metrics_path) as f:
    data = json.load(f)

print("\nHasil evaluasi:")
for key, value in data.items():
    print(f"{key}: {value}")


📊 Ringkasan hasil evaluasi:
metrics/precision(B): 0.7869188119311299
metrics/recall(B): 0.761918862124991
metrics/mAP50(B): 0.8316876460135991
metrics/mAP50-95(B): 0.44876130957241084
fitness: 0.44876130957241084
